In [ ]:
# load and split data

import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("../data/raw/fraud_transactions.csv")
X = df.drop('fraud_flag', axis=1)
y = df['fraud_flag']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# currency conversion
class CurrencyConverter(BaseEstimator, TransformerMixin):
    def __init__(self, rates=None, base_currency="INR"):
        if rates is None:
            rates = {"INR": 1.0, "USD": 83.0, "EUR": 90.0}
        self.rates = rates
        self.base_currency = base_currency

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X["amount_converted"] = X.apply(
            lambda row: row["amount"] * self.rates.get(row["currency"], 1.0), axis=1
        )
        return X
    
    def get_feature_names_out(self, input_features=None):
        """Add 'amount_converted' to the feature names"""
        if input_features is None:
            input_features = []
        # Return original features + new feature
        output_features = list(input_features) + ['amount_converted']
        return np.asarray(output_features, dtype=object)


# typo fixing   
class TypoFixer(BaseEstimator, TransformerMixin):
    def __init__(self, column='merchant_category', typo_map=None):
        if typo_map is None:
            typo_map = {'Groceires': 'Groceries'}
        self.column = column
        self.typo_map = typo_map
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        if self.column in X.columns:
            X[self.column] = X[self.column].replace(self.typo_map)
        return X
    
    def get_feature_names_out(self, input_features=None):
        """Feature names unchanged - just fixing typos"""
        if input_features is None:
            return np.array([self.column], dtype=object)
        return np.asarray(input_features, dtype=object)
    

# outlier clipping
class OutlierClipper(BaseEstimator, TransformerMixin):
    def __init__(self, features=None, lower_quantile=0.01, upper_quantile=0.99):
        self.features = features
        self.lower_quantile = lower_quantile
        self.upper_quantile = upper_quantile
        self.bounds = {}

    def fit(self, X, y=None):
        for col in self.features:
            q_low = X[col].quantile(self.lower_quantile)
            q_high = X[col].quantile(self.upper_quantile)
            self.bounds[col] = (q_low, q_high)
        return self

    def transform(self, X):
        X = X.copy()
        for col, (low, high) in self.bounds.items():
            X[col] = X[col].clip(lower=low, upper=high)
        return X
    
    def get_feature_names_out(self, input_features=None):
        """Feature names unchanged - just clipping values"""
        if input_features is None:
            return np.asarray(self.features, dtype=object)
        return np.asarray(input_features, dtype=object)
    

# Define Feature Groups
num_features = [
    "amount_converted", "velocity", "ip_risk_score", "customer_age",
    "account_tenure", "geo_distance", "merchant_risk_score", "failed_login_attempts"
]

cat_features = [
    "currency", "merchant_category", "transaction_type", "channel", "location"
]

bin_features = ["card_present", "is_international"]


# Pipelines for Each Feature Type

# Numerical pipeline
num_pipeline = Pipeline([
    ("outlier_clipper", OutlierClipper(features=num_features)),
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Categorical pipeline
cat_pipeline = Pipeline([
    ("typo_fixer", TypoFixer()),
    ("imputer", SimpleImputer(strategy="constant", fill_value="Unknown")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

cat_pipeline_catboost = Pipeline([
    ("typo fixer", TypoFixer()),
    ("imputer", SimpleImputer(strategy="constant", fill_value="Unknown"))
])

# Binary pipeline
bin_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent"))
])


# Full Preprocessing Pipeline
preprocessor = Pipeline([
    ("currency_converter", CurrencyConverter()),
    ("transformer", ColumnTransformer([
        ("num", num_pipeline, num_features),
        ("cat", cat_pipeline, cat_features),
        ("bin", bin_pipeline, bin_features)
    ]))
])

preprocessor_catboost = Pipeline([
    ("currency_converter", CurrencyConverter()),
    ("transformer", ColumnTransformer([
        ("num", num_pipeline, num_features),
        ("cat", cat_pipeline_catboost, cat_features),
        ("bin", bin_pipeline, bin_features)
    ]))
])

In [20]:
# Hyperparameter Tuning Example with RandomizedSearchCV

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    "model__n_estimators": [100, 200, 500],
    "model__max_depth": [None, 10, 20],
    "model__min_samples_split": [2, 5, 10]
}

search = RandomizedSearchCV(
    Pipeline([("preprocessor", preprocessor), ("model", RandomForestClassifier())]),
    param_grid,
    scoring="f1",
    cv=3,
    n_iter=10,
    random_state=42
)

search.fit(X_train, y_train)
print("Best params:", search.best_params_)
print("Best score:", search.best_score_)

Best params: {'model__n_estimators': 500, 'model__min_samples_split': 10, 'model__max_depth': None}
Best score: 0.6147265474030242


In [4]:
# Implementing above example model

from sklearn.metrics import f1_score

model = RandomForestClassifier(n_estimators=500, max_depth=None, min_samples_split=10)

testing = Pipeline([
    ("preprocessor", preprocessor),
    ("model", model)
])

testing.fit(X_train, y_train)
y_pred = testing.predict(X_train)
y_prob = testing.predict_proba(X_train)[:,1]

testing.fit(X_test, y_test)
y_pred = testing.predict(X_test)
y_prob = testing.predict_proba(X_test)[:,1]

print("training f1 score:", f1_score(y_train, y_pred))
print("testing f1 score:", f1_score(y_test, y_pred))

f1 score: 0.6149369544131911


In [5]:
from catboost import CatBoostClassifier
from sklearn.model_selection import ParameterSampler, StratifiedKFold
from sklearn.metrics import f1_score
import numpy as np

param_grid_catboost = {
    "iterations": [200, 500],
    "depth": [4, 6, 8],
    "learning_rate": [0.01, 0.1, 0.2],
    "l2_leaf_reg": [1, 3, 5]
}

best_score = -np.inf
best_model = None

# transform training data
X_trf = preprocessor_catboost.fit_transform(X_train)
feature_names = preprocessor_catboost.named_steps["transformer"].get_feature_names_out()
X_train_transformed = pd.DataFrame(X_trf, columns=feature_names)

# identify categorical features for CatBoost
catboost_features = [col for col in X_train_transformed.columns.tolist() if col.startswith("cat_")]

# stratified k-fold cv
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

for params in ParameterSampler(param_grid_catboost, n_iter=5, random_state=42):
    
    cv_scores = []

    for train_index, val_index in skf.split(X_train_transformed, y_train):
        X_train_cv, X_val_cv = X_train_transformed.iloc[train_index], X_train_transformed.iloc[val_index]
        y_train_cv, y_val_cv = y_train.iloc[train_index], y_train.iloc[val_index]

    model = CatBoostClassifier(
        **params,
        loss_function="Logloss",
        eval_metric="F1",
        class_weights=[1, 10],
        random_state=42,
        verbose=0,
        cat_features=[X_train_transformed.columns.get_loc(col) for col in catboost_features]
    )

    model.fit(X_train_cv, y_train_cv)
    y_pred = model.predict(X_val_cv)

    cv_scores.append(f1_score(y_val_cv, y_pred))

    if np.mean(cv_scores) > best_score:
        best_score = np.mean(cv_scores)
        best_model = model

print("Best CatBoost Model:")
print("Best CV F1:", best_score)
print("Best params:", best_model.get_params())


Best CatBoost Model:
Best CV F1: 0.5979381443298969
Best params: {'iterations': 200, 'learning_rate': 0.1, 'depth': 6, 'l2_leaf_reg': 1, 'loss_function': 'Logloss', 'verbose': 0, 'class_weights': [1, 10], 'eval_metric': 'F1', 'random_state': 42, 'cat_features': [8, 9, 10, 11, 12]}


In [6]:
print("Best params:", best_model.get_params())
print(best_model)

Best params: {'iterations': 200, 'learning_rate': 0.1, 'depth': 6, 'l2_leaf_reg': 1, 'loss_function': 'Logloss', 'verbose': 0, 'class_weights': [1, 10], 'eval_metric': 'F1', 'random_state': 42, 'cat_features': [8, 9, 10, 11, 12]}


In [7]:
y_pred_test = best_model.predict(preprocessor_catboost.transform(X_test))

f1_score_test = f1_score(y_test, y_pred_test)
print("Test F1 Score:", f1_score_test)

Test F1 Score: 0.6335877862595419


In [8]:
# CatBoost with overfitting
# That's why must do cross-validation and hyperparameter tuning

try_catboost = CatBoostClassifier(
    iterations=500,
    learning_rate=0.1,
    depth=8,
    l2_leaf_reg=3,
    loss_function="Logloss",
    verbose=0,
    class_weights=[1,10],
    cat_features=[X_train_transformed.columns.get_loc(col) for col in catboost_features],
    random_state=42
)
X_test_transformed = preprocessor_catboost.transform(X_test)

try_catboost.fit(X_train_transformed, y_train)

print("Evaluating Try CatBoost on Test Set...")
y_pred_try = try_catboost.predict(X_test_transformed)
f1_score_try = f1_score(y_test, y_pred_try)
print("Try CatBoost Test F1 Score:", f1_score_try)

print("\n Evaluating Try CatBoost on Train Set...")
y_pred_train_try = try_catboost.predict(X_train_transformed)
f1_score_train_try = f1_score(y_train, y_pred_train_try)
print("Try CatBoost Train F1 Score:", f1_score_train_try)

Evaluating Try CatBoost on Test Set...
Try CatBoost Test F1 Score: 0.6323809523809524

 Evaluating Try CatBoost on Train Set...
Try CatBoost Train F1 Score: 0.9879688605803255


In [9]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import xgboost as xgb
import lightgbm as lgb
import numpy as np

# Define parameter grids
param_grids = {
    "Logistic Regression": {
        "model__C": np.logspace(-3, 3, 7),
        "model__penalty": ["l1", "l2", "elasticnet", None],
        "model__solver": ["saga", "liblinear", "newton-cg", "lbfgs", "sag"]
    },
    "Random Forest": {
        "model__n_estimators": [100, 200, 500],
        "model__max_depth": [None, 10, 20],
        "model__min_samples_split": [2, 5, 10],
        "model__min_samples_leaf": [1, 2, 4]
    },
    "XGBoost": {
        "model__n_estimators": [200, 500],
        "model__max_depth": [3, 6, 10],
        "model__learning_rate": [0.01, 0.1, 0.2],
        "model__subsample": [0.8, 1.0],
        "model__colsample_bytree": [0.8, 1.0]
    },
    "LightGBM": {
        "model__n_estimators": [200, 500],
        "model__num_leaves": [31, 63, 127],
        "model__learning_rate": [0.01, 0.1, 0.2],
        "model__subsample": [0.8, 1.0]
    },
    "AdaBoost": {
        "model__n_estimators": [50, 100, 200],
        "model__learning_rate": [0.01, 0.1, 1.0]
    }
}

# Define models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, class_weight="balanced", random_state=42),
    "Random Forest": RandomForestClassifier(class_weight="balanced", random_state=42),
    "XGBoost": xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss", scale_pos_weight=10, random_state=42),
    "LightGBM": lgb.LGBMClassifier(class_weight="balanced", random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42)
}

results = {}

# Loop through models
for name, model in models.items():
    print(f"Tuning {name}...")
    
    pipe = Pipeline([
        ("preprocessor", preprocessor),
        ("model", model)
    ])
    search = RandomizedSearchCV(
        estimator=pipe,
        param_distributions=param_grids[name],
        n_iter=5,
        scoring="f1",
        cv=3,
        random_state=42
    )
    search.fit(X_train, y_train)
    
    results[name] = {
        "Best Params": search.best_params_,
        "Best Score": search.best_score_
    }

# Show results
import pandas as pd
results_df = pd.DataFrame(results).T
print(results_df)

Tuning Logistic Regression...


d:\Projects\Fraud Detection\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
d:\Projects\Fraud Detection\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
d:\Projects\Fraud Detection\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid thi

Tuning Random Forest...
Tuning XGBoost...


d:\Projects\Fraud Detection\.venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [11:41:38] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\Projects\Fraud Detection\.venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [11:41:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\Projects\Fraud Detection\.venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [11:41:40] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\Projects\Fraud Detection\.venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [11:41:41] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encode

Tuning LightGBM...
[LightGBM] [Info] Number of positive: 952, number of negative: 52381
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001457 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1146
[LightGBM] [Info] Number of data points in the train set: 53333, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive

...

[LightGBM] [Info] Number of positive: 952, number of negative: 52381
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001847 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1146
[LightGBM] [Info] Number of data points in the train set: 53333, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


d:\Projects\Fraud Detection\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 952, number of negative: 52382
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001594 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1146
[LightGBM] [Info] Number of data points in the train set: 53334, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


d:\Projects\Fraud Detection\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 1428, number of negative: 78572
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002472 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1146
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

In [10]:
# best parameters of logistic regression

print("Best parameters for Logistic Regression:")
print(results["Logistic Regression"]["Best Params"])

Best parameters for Logistic Regression:
{'model__solver': 'lbfgs', 'model__penalty': 'l2', 'model__C': np.float64(100.0)}


Class Imbalance Handling
- Logistic Regression, Random Forest, LightGBM, AdaBoost → use class_weight="balanced" or add SMOTE in pipeline.
- XGBoost → use scale_pos_weight (ratio of majority/minority).
- CatBoost → use class_weights=[1,10] (or adjust based on imbalance).


Example with SMOTE for Logistic Regression:


In [11]:
# # Example with SMOTE for Logistic Regression:

# from imblearn.over_sampling import SMOTE
# from imblearn.pipeline import Pipeline as ImbPipeline

# smote = SMOTE(random_state=42)

# log_reg_pipe = ImbPipeline([
#     ("preprocessor", preprocessor),
#     ("smote", smote),
#     ("model", LogisticRegression(
#         max_iter=1000,
#         solver="saga",
#         penalty="elasticnet",
#         C = np.float64(100.0),
#         l1_ratio=0.5, 
#         random_state=42))
# ])

# Feature Importance

In [12]:

log_reg = LogisticRegression(
        max_iter=1000,
        solver="saga",
        penalty="elasticnet",
        C = np.float64(100.0),
        l1_ratio=0.5, 
        random_state=42)

X_trf = preprocessor.fit_transform(X_train)

log_reg.fit(X_trf, y_train)
coef = log_reg.coef_[0]
feature_names = preprocessor.named_steps["transformer"].get_feature_names_out()
feat_imp = pd.Series(coef, index=feature_names).sort_values(key=abs, ascending=False)
print("Logistic Regression Top Features:\n", feat_imp.head(10))

d:\Projects\Fraud Detection\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(


Logistic Regression Top Features:
 cat__merchant_category_Gaming         5.666116
cat__location_Dubai                   5.648972
cat__merchant_category_Luxury         5.500470
cat__merchant_category_Restaurants   -2.444414
cat__merchant_category_Fashion       -2.437944
cat__merchant_category_Groceries     -2.417618
cat__merchant_category_Electronics   -2.238877
cat__merchant_category_Travel        -2.048701
cat__location_Delhi                  -1.366635
cat__location_Bangalore              -1.279190
dtype: float64


d:\Projects\Fraud Detection\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [26]:
import shap

xgb_model = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.1,
    subsample=1.0,
    colsample_bytree=1.0,
    use_label_encoder=False,
    eval_metric="logloss",
    random_state=42
)

explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X_test_transformed)

shap.summary_plot(shap_values, X_test_transformed, feature_names=feature_names)

ImportError: Numba needs NumPy 2.3 or less. Got NumPy 2.4.